See https://irsa.ipac.caltech.edu/docs/program_interface/ztf_metadata.html

- sciimg.fits
- mskimg.fits
- sciimgdaopsfcent.fits
- sciimgdao.psf (only for spatial variation of PSF)
- refimg.fits


```
R filter
l~0,b~0:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0513/459826/ztf_20210513459826_000233_zr_c15_o_q3_sciimg.fits
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0706/370127/ztf_20220706370127_000829_zr_c10_o_q4_sciimg.fits (seeing ~ 1.84453 arcsec)
l~50,b~30:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/1021/119745/ztf_20231021119745_000635_zr_c12_o_q4_sciimg.fits
l~200,b~45:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/0121/340139/ztf_20230121340139_000619_zr_c11_o_q2_sciimg.fits
l~300,b~60:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2020/0306/404421/ztf_20200306404421_000423_zr_c07_o_q1_sciimg.fits
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1120/493183/ztf_20221120493183_000626_zr_c12_o_q2_sciimg.fits (seeing ~ 3.14735 arcsec)
Dwarf galaxy (NGC1569):https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0218/206262/ztf_20210218206262_000811_zr_c16_o_q4_sciimg.fits

I filter
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0831/366910/ztf_20220831366910_000829_zi_c10_o_q4_sciimg.fits (seeing ~ 1.82194 arcsec)
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0301/372245/ztf_20220301372245_000626_zi_c12_o_q2_sciimg.fits (seeing ~ 3.31418 arcsec)

G filter
l~100,b~10:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0901/326748/ztf_20220901326748_000829_zg_c10_o_q4_sciimg.fits (seeing ~ 1.91277 arcsec)
l~150,b~90:https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1126/507882/ztf_20221126507882_000626_zg_c12_o_q2_sciimg.fits (seeing ~ 3.10998 arcsec)


251 ID
l~200, b~ -60: https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2018/0831/500891/ztf_20180831500891_000251_zr_c16_o_q4_sciimg.fits
```

In [ ]:
!git clone https://<YOUR_KEY_HERE>@github.com/Yash-10/deconv_ztf.git

In [ ]:
!cp /kaggle/working/deconv_ztf/*.py /kaggle/working
!cp -r /kaggle/working/deconv_ztf/sgp_reconstruction_results /kaggle/working

# Imports

In [ ]:
!pip install -U photutils
!pip install -U reproject

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
sns.set_style('white')
sns.set_context("paper", font_scale = 2)
sns.set(style='ticks')

import os
import numpy as np
import pandas as pd
import glob
from astropy.io import fits
from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import subprocess

from astropy.io import fits
from astropy.visualization import MinMaxInterval, ImageNormalize, SqrtStretch, ZScaleInterval, PercentileInterval, LogStretch

from scipy.stats import binned_statistic
from scipy.stats import median_abs_deviation

from IPython.display import Image

In [ ]:
!sudo apt-get update
!sudo apt-get install -y source-extractor

In [ ]:
!source-extractor

In [ ]:
# We use Topcat 4.8-7, which is not the latest version.
# Later versions such as v4.8-8/v4.9, for example, may provide different symmetric crossmatching than v4.8-7.
# However, the Topcat/STILTS website (https://www.star.bris.ac.uk/~mbt/topcat/sun253/versions.html) says that one is not necessarily better than the other.
!wget --directory-prefix sgp_reconstruction_results https://www.star.bristol.ac.uk/mbt/releases/topcat/v4.8-7/topcat-full.jar
!wget --directory-prefix sgp_reconstruction_results https://www.star.bris.ac.uk/~mbt/topcat/topcat
%cd sgp_reconstruction_results
!chmod +x topcat
!./topcat -stilts -version
!./topcat -version
%cd ..

# Deconvolution run

For this notebook, uncomment one of the lines in the prefixes below on which deconvolution must be run.

In [ ]:
prefixes = [
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0513/459826/ztf_20210513459826_000233_zr_c15_o_q3_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0706/370127/ztf_20220706370127_000829_zr_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/1021/119745/ztf_20231021119745_000635_zr_c12_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2023/0121/340139/ztf_20230121340139_000619_zr_c11_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2020/0306/404421/ztf_20200306404421_000423_zr_c07_o_q1_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1120/493183/ztf_20221120493183_000626_zr_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2021/0218/206262/ztf_20210218206262_000811_zr_c16_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0831/366910/ztf_20220831366910_000829_zi_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0301/372245/ztf_20220301372245_000626_zi_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/0901/326748/ztf_20220901326748_000829_zg_c10_o_q4_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2022/1126/507882/ztf_20221126507882_000626_zg_c12_o_q2_',
    'https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2018/0831/500891/ztf_20180831500891_000251_zr_c16_o_q4_'
]

image_names = [
    'sciimg.fits',
    'mskimg.fits',
    'sciimgdaopsfcent.fits',
    'sciimgdao.psf',
    # 'refimg.fits'
]

In [ ]:
for prefix in prefixes:
    for image_name in image_names:
        name = prefix + image_name
        result = subprocess.run(['wget', f'{name}'], stdout=subprocess.DEVNULL)
        print(result)

The folder `sgp_reconstruction_results` requires the following files to be present before running `run.py`:

- `default.conv`
- `default.param`
- `default.nnw`
- File provided as argument to `sextractor_config_file_name` below if using SExtractor instead of sep. (NOTE: The sep version is not tested as effectively as the SExtractor version).

In [ ]:
from subprocess import Popen, PIPE, CalledProcessError, STDOUT
def execute(cmd):
    """This function is taken from
    https://stackoverflow.com/a/28319191
    """
#     logfile = open('sgp_run_all.log', 'a')
    with Popen(cmd, stdout=PIPE, bufsize=1, universal_newlines=True, stderr=STDOUT) as p:
        for line in p.stdout:
            print(line, end='') # process line here

    if p.returncode != 0:
        raise CalledProcessError(p.returncode, p.args)

for prefix in prefixes:
    command = [
        'python3', 'run.py', '--data_path_sciimg', f'/kaggle/working/{prefix.split("/")[-1]}sciimg.fits',
        '--data_path_psf', f'/kaggle/working/{prefix.split("/")[-1]}sciimgdaopsfcent.fits',
        '--init_recon', '2', '--stop_criterion', '3', '--use_subdiv', '--subdiv_size', '512', '--subdiv_overlap', '10',
        '--use_sextractor', '--sextractor_config_file_name', 'sextractor_config.fits_scat_sextractor.sex',
        '--use_subdiv', '--add_bkg_to_deconvolved', '--interpolate_bad_pixels',
        '--tol_convergence', '1e-4', '--reconstruct_full_image_from_subdivisions',
        # '--pixel_mask', '/kaggle/working/ztf_20180831500891_000251_zr_c16_o_q4_mskimg.fits',  # todo: how to use this?
        '--perform_catalog_crossmatching'
    ]

    execute(command)

## Outro

Need to upload kaggle.json file manually.

In [ ]:
# !chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# !mv kaggle.json /root/.kaggle

In [ ]:
# !kaggle datasets init -p /kaggle/working/sgp_reconstruction_results

In [ ]:
# !kaggle datasets create -p /kaggle/working/sgp_reconstruction_results

In [ ]:
# !mv kaggle.json /root/.kaggle

In [1]:
!kaggle kernels output yashg1002/ztf-deconvolution-run -p /kaggle/working/ztf_deconv_results

Traceback (most recent call last):
  File "/opt/conda/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/opt/conda/lib/python3.10/site-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/opt/conda/lib/python3.10/site-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
